In [2]:
from causallib.datasets import load_nhefs
from causallib.estimation import IPW, PropensityMatching, StratifiedStandardization
from causallib.evaluation import evaluate
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression

In [3]:
from causallib.datasets import load_nhefs

# ============================================================
# NHEFS データセットの読み込み
# ============================================================
# load_nhefs() は、causallib に同梱されている
#   NHEFS (National Health and Nutrition Examination Follow-up Study)
# の禁煙 × 体重変化データを読み込むユーティリティ関数。
#
# このデータは Hernán & Robins『Causal Inference: What If』の
# 典型例としても使われている、因果推論の「教科書的」データセットで、
#
#   - 対象：1970–80年代のコホート
#   - 介入（treatment）: 喫煙をやめたか否か（quit smoking）
#   - アウトカム（outcome）: 1971–1982 年の体重変化（wt82_71）
#   - 共変量（covariates）: 年齢・性別・喫煙歴・健康状態など多数
#
# といった構造を持つ「観察研究デザイン」のデータである。
#
# causallib の load_nhefs() は、単なる DataFrame ではなく
# sklearn の Bunch に似たオブジェクトを返し、
#
#   data.X : 共変量（交絡因子を含む特徴量）の DataFrame
#   data.a : 介入変数（禁煙したかどうか, 0/1）の Series
#   data.y : 観測アウトカム（体重差 wt82_71）の Series
#   data.po: 可能性アウトカム（ポテンシャルアウトカム）への擬似的な真値
#   data.descriptors : 各変数の意味に関するメタデータ
#
# といった属性にアクセスできる。
#
# 理論的な位置づけ：
# - この一行で「IPW（逆確率重み付け）」「標準化」「ダブルロバスト推定」など、
#   さまざまな因果推定アルゴリズムを試せる実験用データが手に入る。
# - 特に、data.X をバックドア調整集合として扱い、
#   data.a を処置変数、data.y をアウトカムとして ATE/ATT を推定する、
#   というポテンシャルアウトカム・フレームワークの典型的な流れを
#   そのままコードで体験できる。
data = load_nhefs()

In [4]:
# ============================================================
# 傾向スコア（propensity score）モデルの定義
# ============================================================
# 傾向スコア e(X) は、
#
#   e(X) = P(A = 1 | X)
#
# のことを指し、「共変量 X を持つ個体が、介入 A=1 を受ける条件付き確率」を表す。
# ポテンシャルアウトカム・フレームワークでは、
#
#   - A: 処置（例：禁煙したかどうか）
#   - Y: アウトカム（例：体重変化）
#   - X: 共変量（交絡因子を含む要因）
#
# としたとき、交絡の制御を行うために e(X) を推定し、
# その値を用いてマッチング・IPW・層化などを行うことで、
# 「A と X が独立になるような擬似ランダム化」を実現しようとする。
#
# ここでは、その e(X) を **ロジスティック回帰モデル**で推定している。
# ============================================================

# 傾向スコアを算出するためのロジスティック回帰モデルを定義
learner = LogisticRegression(
    # solver は最適化アルゴリズムの種類を指定する引数。
    #
    # - "liblinear" は L2 正則化つきロジスティック回帰の古典的な解法の一つで、
    #   特にサンプル数や特徴量数がそこまで巨大でない場合に安定して動作しやすい。
    # - 小〜中規模データセットでの二値分類（今回のような A∈{0,1}）には十分であり、
    #   収束性や数値安定性の点からも扱いやすい設定。
    solver="liblinear",
    # class_weight="balanced" は、クラス不均衡（例：禁煙者が少ない／多い）の補正のための設定。
    #
    # - 通常、ロジスティック回帰は「サンプル数の多いクラス」に引きずられやすい。
    #   例えば A=0 が 90%、A=1 が 10% のような場合、
    #   何もしないと「ほとんど A=0 と予測してしまう」モデルでも
    #   損失がそれなりに小さくなってしまう。
    #
    # - class_weight="balanced" とすることで、
    #   各クラスの重みを 1 / (サンプル数 × クラス頻度) に比例させ、
    #   事実上「少数クラスを重く扱う」ようにする。
    #
    # 傾向スコア推定の観点では、
    # - P(A=1 | X), P(A=0 | X) のどちらも適切に学習されないと
    #   IPW やマッチング時に極端な重みが出やすくなる。
    # - そのため、クラス不均衡があるケースでは class_weight を調整して
    #   推定の安定性を高めるのは理にかなっている。
    class_weight="balanced",
)

# ============================================================
# 傾向スコアの算出とマッチング（Propensity Matching）
# ============================================================
# PropensityMatching は causallib が提供する「傾向スコアマッチング」のクラス。
#
# 理論的なアイデア：
# - e(X) = P(A=1 | X) を推定したあと、
#   「処置群 A=1 の個体」と「対照群 A=0 の個体」を
#   傾向スコア e(X) が近いペア同士でマッチングする。
#
# - うまくマッチングできれば、マッチされたサンプル集合においては
#   共変量 X と処置 A がほぼ独立になり、
#   「ランダム化実験に近い状況」が得られると期待される。
#
# - その上で、マッチされたペア間のアウトカム Y の差を平均することで、
#   A の Y に対する因果効果（例えば ATT: 処置群平均処置効果）を推定する。
#
# ここでは、先ほど定義したロジスティック回帰 learner を
# 「傾向スコア推定器」として PropensityMatching に渡している。
pm = PropensityMatching(learner=learner)

# ------------------------------------------------------------
# モデルの学習（fit）
# ------------------------------------------------------------
# pm.fit(X, a, y) の引数の意味：
#
# - data.X : 共変量（交絡因子を含む特徴量）
#   → 傾向スコア e(X) の説明変数になる。
#
# - data.a : 処置変数（例：禁煙したかどうか, 0/1）
#   → 傾向スコアの目的変数（ロジスティック回帰のラベル）。
#
# - data.y : アウトカム（例：体重変化）
#   → マッチング後に因果効果を評価するために用いられる。
#
# fit の内部で行われること（概念的な流れ）：
#   1. learner（ロジスティック回帰）で e_hat(X) ≈ P(A=1 | X) を推定
#   2. e_hat(X) を用いて A=1 と A=0 の個体をマッチング
#   3. マッチング結果（インデックスのペアや重み）を内部に保持
#
# この段階ではまだ「因果効果（ATE/ATT）」自体は計算していないが、
# 後続で pm.estimate_population_outcome(...), pm.estimate_effect(...),
# あるいは causallib の標準 API を用いることで、
# 実際の効果推定に進むことができる。
pm.fit(data.X, data.a, data.y)

PropensityMatching(caliper=None, covariance_conditioner=EmpiricalCovariance(), estimate_observed_outcome=False, knn_backend=sklearn, matching_mode=both, metric=mahalanobis, n_neighbors=1, propensity_transform=PropensityTransformer(learner=LogisticRegression(class_weight='balanced',
                                                 solver='liblinear')), with_replacement=True,
                   learner=LogisticRegression(class_weight='balanced', solver='liblinear'))